In [2]:
with open('chinesename.txt', 'r') as f:
    name = f.read().decode('utf-8').split()
    
    
import requests
from requests import exceptions
from socket import timeout, error
from bs4 import BeautifulSoup
import time
from fake_useragent import UserAgent
import random
import math
import re


def randomName(name):
    indexN = random.randint(0, len(name)-1)
    result = name[indexN]
    return result
def randomUid():
    uid = ''
    for ele in xrange(16):
        if not uid:
            uid += '1'
        else:
            uid += str(random.randint(0,9))
    return uid
def count(soup):
    buttons = soup.findAll(attrs={"type" : "submit"})
    button = [ele['value'] for ele in buttons if "Download" in ele['value']][0]
    size = int(re.search('.*\((\d+)\).*', button).group(1))
    pages = int(math.ceil(size/30.0))
    return pages
def proxyList(n, Uptime = 0):
    result = []
    url = 'http://gatherproxy.com/proxylist/anonymity/?t=elite'
    ua = UserAgent()
    limit = n
    for index in xrange(n):
        header = {
            "User-Agent":ua.chrome
        }
        data =  {
            "Type":"elite",
            "PageIdx":str(index+1),
            "Uptime":str(Uptime)
        }
        if limit >= index+1:
            try:
                r = requests.post(url, headers=header, data=data, timeout=5)
                soup = BeautifulSoup(r.text, 'html.parser')
                limit = count(soup)
                temp = ''.join([ele.text for ele in soup.select('script') if 'document.write' in ele.text])
                templist = temp.replace('))',')),')[0:-1].split(',')
                for ele in templist:
                    ipandport = re.search("(\d+\.\d+\.\d+\.\d+).*gp.dep.*\'(.*)\'",ele)
                    proxy = "http://%s:%d" %(ipandport.group(1), int(ipandport.group(2),16))
                    result.append(proxy)
            except (exceptions.Timeout, exceptions.ConnectionError):
                pass
        else:
            break
    return result
def randomProxy(proxyList):
    randindex = random.randint(0,len(proxyList)-1)
    return {"http":proxyList[randindex]}
def vote(number):
    limit = int(math.ceil(number/30.0)) if number <= 1500 else 50
    pxlist = proxyList(limit, Uptime = random.randint(50,100))
    n = 1
    e = 0
    while n <= number:
        px = {}
        ua = UserAgent()
        payload = {
            "uid":randomUid(),
            "cname":randomName(name),
            "post_id":"8701"
        }
        header = {
            "User-Agent":ua.chrome
        }
        try:
            if not pxlist:
                leverage = e * (e+n) / float(n) if e != 0 else 1
                adj = int(math.ceil(leverage/30))
                pxlist = proxyList(adj, Uptime = random.randint(0,49))
            r = random.randint(0,len(pxlist)-1)
            px = {'http':pxlist[r]}
            pxlist.pop(r)             
            res = requests.post("http://www.2016catwalkcompetition.com.tw/wp-content/themes/scent/check-user.php", proxies=px, data = payload, headers = header, timeout=5)
            n += 1
            #print px['http'], res.text
        except (exceptions.Timeout, exceptions.ProxyError, exceptions.ConnectionError, timeout, error):
            e += 1
            #print px['http'], "Proxy Error"
        time.sleep(random.randint(1,5))

vote(10)